In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

from setting import *

In [ ]:
path_dict = path(
    NAME,
    ELEMENTS,
    NMF_K,
    WT_HCC_K,
    H_HCC_K,
    UPLOAD_TO_PLOTLY,
)

In [ ]:
gps_map = ccal.load_gps_map(path_dict['gps_map_file_path'])

In [ ]:
feature_dicts = make_feature_dicts()

In [ ]:
with open('../code/highlight.json') as json_file:
    
    highlight = json.load(json_file)
    
highlight

In [ ]:
for wt_or_h, element_feature_dicts, element_highlight, element_marker_size in (
    (
        'h',
        feature_dicts['Sample'],
        highlight['Sample'],
        GPS_MAP_H_ELEMENT_MARKER_SIZE,
    ),
):
    
    for feature_group, features in element_highlight.items():
        
        if feature_group in element_feature_dicts:
            
            df = element_feature_dicts[feature_group]['df']
            
            annotation_type = element_feature_dicts[feature_group]['data_type']
            
            for feature in features:
                
                if feature in df.index:
                    
                    maps_plotly_directory_path = path_dict['{}_map_plotly_directory_path'.format(wt_or_h)]
                    
                    if maps_plotly_directory_path is None:
                        
                        plotly_file_path = None
                        
                    else:
                        
                        plotly_file_path = '{}/{} {}'.format(
                            maps_plotly_directory_path,
                            feature_group,
                            feature,
                        )
                        
                    element_values = df.loc[feature]
                    
                    title = '({}) {}'.format(
                        feature_group,
                        feature,
                    )
                    
                    gps_map.plot_gps_map(
                        wt_or_h,
                        annotation_x_element=element_values.to_frame().T,
                        annotation_std_maxs=None,
                        annotation_types=(annotation_type, ),
                        title=title,
                        element_marker_size=element_marker_size,
                        html_file_path='{}/{} {}'.format(
                            path_dict['{}_map_directory_path'.format(wt_or_h)],
                            feature_group,
                            feature,
                        ),
                        plotly_file_path=plotly_file_path,
                    )
                    
                    if gps_map.h_element_states is not None:
                        
                        ccal.plot_violin_or_box(
                            list(values for group, values in element_values.groupby(by=gps_map.h_element_states)),
                            names=tuple('State {}'.format(i) for i in sorted(set(gps_map.h_element_states))),
                            violin_or_box='box',
                            title=title,
                        )